**This transcription task is modified from below notebook and run on kaggle**

# **Youtube Videos Transcription with OpenAI's Whisper**

[![blog post shield](https://img.shields.io/static/v1?label=&message=Blog%20post&color=blue&style=for-the-badge&logo=openai&link=https://openai.com/blog/whisper)](https://openai.com/blog/whisper)
[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/openai/whisper)](https://github.com/openai/whisper)
[![paper shield](https://img.shields.io/static/v1?label=&message=Paper&color=blue&style=for-the-badge&link=https://cdn.openai.com/papers/whisper.pdf)](https://cdn.openai.com/papers/whisper.pdf)
[![model card shield](https://img.shields.io/static/v1?label=&message=Model%20card&color=blue&style=for-the-badge&link=https://github.com/openai/whisper/blob/main/model-card.md)](https://github.com/openai/whisper/blob/main/model-card.md)

Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification.

This Notebook will guide you through the transcription of a Youtube video using Whisper. You'll be able to explore most inference parameters or use the Notebook as-is to store the transcript and video audio in your Google Drive.

In [1]:
#@markdown # **Check GPU type** 🕵️

#@markdown The type of GPU you get assigned in your Colab session defined the speed at which the video will be transcribed.
#@markdown The higher the number of floating point operations per second (FLOPS), the faster the transcription.
#@markdown But even the least powerful GPU available in Colab is able to run any Whisper model.
#@markdown Make sure you've selected `GPU` as hardware accelerator for the Notebook (Runtime &rarr; Change runtime type &rarr; Hardware accelerator).

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---
#@markdown **Factory reset your Notebook's runtime if you want to get assigned a new GPU.**

!nvidia-smi -L

!nvidia-smi

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-25168850-046e-9417-8a1e-aae5fdad47be)
Sun Jun 22 00:21:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                    

In [2]:
#@markdown # **Install libraries** 🏗️
#@markdown This cell will take a little while to download several libraries, including Whisper.

#@markdown ---

! pip install git+https://github.com/openai/whisper.git
! pip install yt-dlp

import sys
import warnings
import whisper
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-y_g6qv81
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-y_g6qv81
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━

Using device: cuda:0


In [3]:
#@markdown # **Model selection** 🧠

#@markdown As of the first public release, there are 4 pre-trained options to play with:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
#@markdown | large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

#@markdown ---
Model = 'medium' #@param ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large']
#@markdown ---
#@markdown **Run this cell again if you change the model.**

whisper_model = whisper.load_model(Model)

if Model in whisper.available_models():
    display(Markdown(
        f"**{Model} model is selected.**"
    ))
else:
    display(Markdown(
        f"**{Model} model is no longer available.**<br /> Please select one of the following:<br /> - {'<br /> - '.join(whisper.available_models())}"
    ))

100%|█████████████████████████████████████| 1.42G/1.42G [00:32<00:00, 47.3MiB/s]


**medium model is selected.**

In [4]:
def transcribe_one_video(url):
    #@markdown # **Video selection** 📺

    #@markdown Enter the URL of the Youtube video you want to transcribe, wether you want to save the audio file in your Google Drive, and run the cell.
    
    Type = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive']
    #@markdown ---
    #@markdown #### **Youtube video or playlist**
    URL = url #@param {type:"string"}
    # store_audio = True #@param {type:"boolean"}
    #@markdown ---
    #@markdown #### **Google Drive video, audio (mp4, wav), or folder containing video and/or audio files**
    video_path = "Colab Notebooks/transcription/my_video.mp4" #@param {type:"string"}
    #@markdown ---
    #@markdown **Run this cell again if you change the video.**
    
    video_path_local_list = []
    
    if Type == "Youtube video or playlist":
    
        ydl_opts = {
            'format': 'm4a/bestaudio/best',
            'outtmpl': 'temp.%(ext)s',
            # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
            'postprocessors': [{  # Extract audio using ffmpeg
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
            }]
        }
    
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            error_code = ydl.download([URL])
            list_video_info = [ydl.extract_info(URL, download=False)]
    
        for video_info in list_video_info:
            video_path_local_list.append(Path(f"temp.wav"))
    
    elif Type == "Google Drive":
        # video_path_drive = drive_mount_path / Path(video_path.lstrip("/"))
        video_path = drive_mount_path / Path(video_path.lstrip("/"))
        if video_path.is_dir():
            for video_path_drive in video_path.glob("**/*"):
                if video_path_drive.is_file():
                    display(Markdown(f"**{str(video_path_drive)} selected for transcription.**"))
                elif video_path_drive.is_dir():
                    display(Markdown(f"**Subfolders not supported.**"))
                else:
                    display(Markdown(f"**{str(video_path_drive)} does not exist, skipping.**"))
                video_path_local = Path(".").resolve() / (video_path_drive.name)
                shutil.copy(video_path_drive, video_path_local)
                video_path_local_list.append(video_path_local)
        elif video_path.is_file():
            video_path_local = Path(".").resolve() / (video_path.name)
            shutil.copy(video_path, video_path_local)
            video_path_local_list.append(video_path_local)
            display(Markdown(f"**{str(video_path)} selected for transcription.**"))
        else:
            display(Markdown(f"**{str(video_path)} does not exist.**"))
    
    else:
        raise(TypeError("Please select supported input type."))
    
    for video_path_local in video_path_local_list:
        if video_path_local.suffix == ".mp4":
            video_path_local = video_path_local.with_suffix(".wav")
            result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])

    #@markdown # **Run the model** 🚀

    #@markdown Run this cell to execute the transcription of the video. This can take a while and very based on the length of the video and the number of parameters of the model selected above.
    
    #@markdown ## **Parameters** ⚙️
    
    #@markdown ### **Behavior control**
    #@markdown ---
    language = "Auto detection" #@param ['Auto detection', 'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Azerbaijani', 'Bashkir', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Breton', 'Bulgarian', 'Burmese', 'Castilian', 'Catalan', 'Chinese', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Estonian', 'Faroese', 'Finnish', 'Flemish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Lao', 'Latin', 'Latvian', 'Letzeburgesch', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Moldavian', 'Moldovan', 'Mongolian', 'Myanmar', 'Nepali', 'Norwegian', 'Nynorsk', 'Occitan', 'Panjabi', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Pushto', 'Romanian', 'Russian', 'Sanskrit', 'Serbian', 'Shona', 'Sindhi', 'Sinhala', 'Sinhalese', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tibetan', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uzbek', 'Valencian', 'Vietnamese', 'Welsh', 'Yiddish', 'Yoruba']
    #@markdown > Language spoken in the audio, use `Auto detection` to let Whisper detect the language.
    #@markdown ---
    verbose = 'Progress bar' #@param ['Live transcription', 'Progress bar', 'None']
    #@markdown > Whether to print out the progress and debug messages.
    #@markdown ---
    output_format = 'txt' #@param ['txt', 'vtt', 'srt', 'tsv', 'json', 'all']
    #@markdown > Type of file to generate to record the transcription.
    #@markdown ---
    task = 'transcribe' #@param ['transcribe', 'translate']
    #@markdown > Whether to perform X->X speech recognition (`transcribe`) or X->English translation (`translate`).
    #@markdown ---
    
    #@markdown <br/>
    
    #@markdown ### **Optional: Fine tunning**
    #@markdown ---
    temperature = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}
    #@markdown > Temperature to use for sampling.
    #@markdown ---
    temperature_increment_on_fallback = 0.2 #@param {type:"slider", min:0, max:1, step:0.05}
    #@markdown > Temperature to increase when falling back when the decoding fails to meet either of the thresholds below.
    #@markdown ---
    best_of = 5 #@param {type:"integer"}
    #@markdown > Number of candidates when sampling with non-zero temperature.
    #@markdown ---
    beam_size = 8 #@param {type:"integer"}
    #@markdown > Number of beams in beam search, only applicable when temperature is zero.
    #@markdown ---
    patience = 1.0 #@param {type:"number"}
    #@markdown > Optional patience value to use in beam decoding, as in [*Beam Decoding with Controlled Patience*](https://arxiv.org/abs/2204.05424), the default (1.0) is equivalent to conventional beam search.
    #@markdown ---
    length_penalty = -0.05 #@param {type:"slider", min:-0.05, max:1, step:0.05}
    #@markdown > Optional token length penalty coefficient (alpha) as in [*Google's Neural Machine Translation System*](https://arxiv.org/abs/1609.08144), set to negative value to uses simple length normalization.
    #@markdown ---
    suppress_tokens = "-1" #@param {type:"string"}
    #@markdown > Comma-separated list of token ids to suppress during sampling; '-1' will suppress most special characters except common punctuations.
    #@markdown ---
    initial_prompt = "" #@param {type:"string"}
    #@markdown > Optional text to provide as a prompt for the first window.
    #@markdown ---
    condition_on_previous_text = True #@param {type:"boolean"}
    #@markdown > if True, provide the previous output of the model as a prompt for the next window; disabling may make the text inconsistent across windows, but the model becomes less prone to getting stuck in a failure loop.
    #@markdown ---
    fp16 = True #@param {type:"boolean"}
    #@markdown > whether to perform inference in fp16.
    #@markdown ---
    compression_ratio_threshold = 2.4 #@param {type:"number"}
    #@markdown > If the gzip compression ratio is higher than this value, treat the decoding as failed.
    #@markdown ---
    logprob_threshold = -1.0 #@param {type:"number"}
    #@markdown > If the average log probability is lower than this value, treat the decoding as failed.
    #@markdown ---
    no_speech_threshold = 0.6 #@param {type:"slider", min:-0.0, max:1, step:0.05}
    #@markdown > If the probability of the <|nospeech|> token is higher than this value AND the decoding has failed due to `logprob_threshold`, consider the segment as silence.
    #@markdown ---
    
    verbose_lut = {
        'Live transcription': True,
        'Progress bar': False,
        'None': None
    }
    
    args = dict(
        language = (None if language == "Auto detection" else language),
        verbose = verbose_lut[verbose],
        task = task,
        temperature = temperature,
        temperature_increment_on_fallback = temperature_increment_on_fallback,
        best_of = best_of,
        beam_size = beam_size,
        patience=patience,
        length_penalty=(length_penalty if length_penalty>=0.0 else None),
        suppress_tokens=suppress_tokens,
        initial_prompt=(None if not initial_prompt else initial_prompt),
        condition_on_previous_text=condition_on_previous_text,
        fp16=fp16,
        compression_ratio_threshold=compression_ratio_threshold,
        logprob_threshold=logprob_threshold,
        no_speech_threshold=no_speech_threshold
    )
    
    temperature = args.pop("temperature")
    temperature_increment_on_fallback = args.pop("temperature_increment_on_fallback")
    if temperature_increment_on_fallback is not None:
        temperature = tuple(np.arange(temperature, 1.0 + 1e-6, temperature_increment_on_fallback))
    else:
        temperature = [temperature]
    
    if Model.endswith(".en") and args["language"] not in {"en", "English"}:
        warnings.warn(f"{Model} is an English-only model but receipted '{args['language']}'; using English instead.")
        args["language"] = "en"
    
    for video_path_local in video_path_local_list:
        display(Markdown(f"### {video_path_local}"))
    
        video_transcription = whisper.transcribe(
            whisper_model,
            str(video_path_local),
            temperature=temperature,
            **args,
        )
    
        # Save output
        whisper.utils.get_writer(
            output_format=output_format,
            output_dir=video_path_local.parent
        )(
            video_transcription,
            str(video_path_local.stem),
            options=dict(
                highlight_words=False,
                max_line_count=None,
                max_line_width=None,
            )
        )
    
        # def exportTranscriptFile(ext: str):
        #     # local_path = video_path_local.parent / video_path_local.with_suffix(ext).name
        #     local_path = Path("temp" + ext)
        #     # export_path = drive_whisper_path / video_path_local.with_suffix(ext).name
        #     # shutil.copy(
        #     #     local_path,
        #     #     export_path
        #     # )
        #     display(Markdown(f"**Transcript file created: {local_path}**"))
    
        # if output_format=="all":
        #     for ext in ('.txt', '.vtt', '.srt', '.tsv', '.json'):
        #         exportTranscriptFile(ext)
        # else:
        #     exportTranscriptFile("." + output_format)



In [5]:
# read txt files and get urls
with open("/kaggle/input/tedxps-data/tedxps_urls.txt", "r") as file:
    urls = [line.strip() for line in file.readlines() if line.strip()]

transcripts = {}
start = 120

# loop through urls
for i, url in enumerate(urls):
    if i+1 < start:
        continue

    # if no youtube urls
    if url == "-":
        transcripts[i+1] = "-"
        continue

    transcribe_one_video(url)
    
    # store transcript in dict with its id matching database
    with open("temp.txt", "r") as transcript_file:
        transcripts[i+1] = transcript_file.read()

[youtube] Extracting URL: https://www.youtube.com/watch?v=mFSpxIzJ2Hw
[youtube] mFSpxIzJ2Hw: Downloading webpage
[youtube] mFSpxIzJ2Hw: Downloading tv client config
[youtube] mFSpxIzJ2Hw: Downloading player 9fe2e06e-main
[youtube] mFSpxIzJ2Hw: Downloading tv player API JSON
[youtube] mFSpxIzJ2Hw: Downloading ios player API JSON
[youtube] mFSpxIzJ2Hw: Downloading m3u8 information
[info] mFSpxIzJ2Hw: Downloading 1 format(s): 140
[download] Destination: temp.m4a
[download] 100% of   15.59MiB in 00:00:06 at 2.36MiB/s     
[FixupM4a] Correcting container of "temp.m4a"
[ExtractAudio] Destination: temp.wav
Deleting original file temp.m4a (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=mFSpxIzJ2Hw
[youtube] mFSpxIzJ2Hw: Downloading webpage
[youtube] mFSpxIzJ2Hw: Downloading tv client config
[youtube] mFSpxIzJ2Hw: Downloading tv player API JSON
[youtube] mFSpxIzJ2Hw: Downloading ios player API JSON
[youtube] mFSpxIzJ2Hw: Downloading m3u8 information


### temp.wav

Detected language: Russian


 93%|█████████▎| 94290/101006 [13:57<00:59, 112.53frames/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=hqh59ovGWfY
[youtube] hqh59ovGWfY: Downloading webpage


...

[youtube] Extracting URL: https://www.youtube.com/watch?v=_B3KvfNTVq4
[youtube] _B3KvfNTVq4: Downloading webpage


[youtube] _B3KvfNTVq4: Downloading tv client config
[youtube] _B3KvfNTVq4: Downloading tv player API JSON
[youtube] _B3KvfNTVq4: Downloading ios player API JSON
[youtube] _B3KvfNTVq4: Downloading m3u8 information
[info] _B3KvfNTVq4: Downloading 1 format(s): 140
[download] Destination: temp.m4a
[download] 100% of    7.49MiB in 00:00:03 at 2.42MiB/s   
[FixupM4a] Correcting container of "temp.m4a"
[ExtractAudio] Destination: temp.wav
Deleting original file temp.m4a (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=_B3KvfNTVq4
[youtube] _B3KvfNTVq4: Downloading webpage
[youtube] _B3KvfNTVq4: Downloading tv client config
[youtube] _B3KvfNTVq4: Downloading tv player API JSON
[youtube] _B3KvfNTVq4: Downloading ios player API JSON
[youtube] _B3KvfNTVq4: Downloading m3u8 information


### temp.wav

Detected language: Chinese


100%|██████████| 48513/48513 [01:45<00:00, 460.80frames/s]


In [6]:
# store transcripts in json file
import json

with open("tedxps_transcripts.json", "w", encoding="utf-8") as f:
    json.dump(transcripts, f, indent=4, ensure_ascii=False)